In [252]:
#import libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import csv 
import lxml
from textblob import TextBlob
import regex as re

In [310]:
# get raw html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
soup = BeautifulSoup(gmo.content, "lxml")
# get text 
souptext = str(soup.get_text())

In [305]:
# get rid of leading and trailing whitespace
genetext_raw = souptext.splitlines() 

# remove leading and trailing whitespace
genetext = []
for element in genetext_raw: 
    genetext_entry = element.strip()
    genetext.append(genetext_entry)

# remove empty list entries 
while('' in genetext) : 
    genetext.remove('') 

# slice out only parts we want
genetext = genetext[34:-15]
print(genetext)

['Genes List', 'Gene', 'Product and Function', 'Trait: 2,4-D herbicide tolerance', 'aad-1', 'Source:', 'synthetic form of the aad-1 gene from Sphingobium herbicidovorans', 'Product:', 'aryloxyalkanoate dioxygenase 1 (AAD-1) protein', 'Function:', 'detoxifies 2,4-D herbicide by side-chain degradation and degrades the R-enantiomers of aryloxyphenoxypropionate herbicides', 'aad-12', 'Source:', 'Delftia acidovorans', 'Product:', 'aryloxyalkanoate di-oxygenase 12 (AAD-12) protein', 'Function:', 'catalyzes the side chain degradation of 2,4-D herbicide', 'Trait: Altered lignin production', 'ccomt (inverted repeat)', 'Source:', 'Medicago sativa (alfalfa)', 'Product:', 'dsRNA that suppresses endogenous S-adenosyl-L-methionine: trans-caffeoyl CoA 3-O-methyltransferase (CCOMT gene) RNA transcript levels via the RNA interference (RNAi) pathway', 'Function:', 'reduces content of guaiacyl (G) lignin', 'EgCAld5H', 'Source:', 'Eucalyptus grandis', 'Product:', 'CAld5H enzyme', 'Function:', 'regulates t

In [ ]:
# iterate, +1 index for keyword? (trait inclusive, gene name is -1 source:)

In [319]:
# using blobtext
souptext = str(soup.get_text())
blobtext = TextBlob(souptext)
blobtext2 = blobtext.strip()
print(blobtext2, file=open("blobtext.txt", "a")) # write to file

In [315]:
# dictionary for regex matching
gene_dict = {
    'trait': re.compile(r'Trait: (?P<trait>.*)\n'),
    'gene' : re.compile(r'\n(?P<gene>.*\s)\n Source:'),
    'source': re.compile(r'Source:\n(?P<source>.*)\n'), 
    'product': re.compile(r'Product:\n(?P<product>.*)\n'),
    'function': re.compile(r'Function:\n(?P<function>.*)\n')
}

In [316]:
# match lines to regex
def parseinfo(text): 
    for info, regex in gene_dict.items():
        match = regex.search(text)
        if match: 
            return info, match
    return None, None

In [331]:
# parse blobtext file 


######### needs debugging!! 
def parsefile(file): 
    data = []
    with open(file, 'r') as file: 
        line = file.readline()
        while line: 
            info, match = parseinfo(line)
            if info == 'trait': 
                trait = match.group('trait')
            if info == 'gene': 
                gene = match.group('gene')
            if info == 'source': 
                source = match.group('source')
            if info == 'product':
                product = match.group('product')
            if info == 'function': 
                function = match.group('function')
                
            while line.strip():
                row = {
                    'Trait': trait, 
                    'Gene': gene,
                    'Source': source,
                    'Product': product,
                    'Function': function
                }
                
                data.append(row)
                line = file.readline() #?? 
                
            line = file.readline()

        data = pd.DataFrame(data)
        data.set_index(['Trait', 'Gene'], inplace = True)
        data = data.groupby(level = data.index.names).first()
    return data

In [332]:
parsefile('./blobtext.txt')

UnboundLocalError: local variable 'trait' referenced before assignment

## Scrapwork

In [220]:
str_gt = str(genetext_raw)
df = pd.DataFrame([x.split(',') for x in str_gt.split('\n')])
pd.DataFrame.dropna(df, how = 'all')

,0,1,2,3,4,5,6,7,8,9,...,2572,2573,2574,2575,2576,2577,2578,2579,2580,2581
0,['','','','','',' ',' ','','Genes List - GM Approval Database | ISAAA.org','',...,'',""" ga('create'",'UA-11798562-1',"'auto');""",""" ga('send'","'pageview');""",'','','','']


In [ ]:
gene_names = []
gmo_tds = soup("tr", class_="smalltext")
for td in gmo_tds:
    gene_names.append(td.find("a").contents[0])
gene_names

In [ ]:
soup = BeautifulSoup('<td bgcolor="#F7F7F7" valign="top"><strong>Product:</strong> CAld5H enzyme <br/>')
tag = soup.b
type(tag)

In [ ]:
classes = []
for element in soup.find_all(class_=True):
    classes.extend(element["class"])
print(classes)